[//]: # "Para temas claros usar logo_utn.svg"
<img src="./logo_utn_dark.png" align="right" width="250" /> 

# Análisis simbólico de la transferencia en la topología Sallen-Key

#### Por Mariano Llamedo Soria

## Resumen

En este documento se analizará un filtro pasabajos Sallen-Key, obteniendo su función transferencia y analizando sus parámetros críticos mediantes el estudio de la sensibilidad. Luego se profundiza este análisis mediante una simulacion numérica y circuital de tipo Montecarlo, para finalmente extraer algunas conclusiones.


<a id='indice'></a>


## Índice

El siguiente índice permitirá navegar por las secciones del documento con mayor facilidad.


1. [Introducción](#intro)


2. Simulación simbólica:

    a. [Análisis teórico de la función transferencia](#1.a).

    b. [Asumiendo realimentación negativa (RN) con opamp ideal](#1.b).
    
    c. [Sin asumir RN con opamp ideal](#1.c).
    
    d. [Sin asumir RN y con opamp NO ideal](#1.d).
    
    e. [Análisis de sensibilidad](#1.e).
 



<a id='intro'></a>

### I) Introducción


En el siguiente documento se analizará la dispersión de parámetros:

 * Factor de selectividad ($Q$)
 * Frecuencia de corte o resonancia ($\omega_0$)
 * y la ganancia ($K$)

correspondiente a una topología Sallen-Key en su configuración pasabajo, mediante la siguiente red:


<img src="./sallenkey.png" align="center" width="700" style="border:10px solid white" /> 

Este circuito implementa la siguiente transferencia de tensión:

$$ T(s) =\frac{V_o}{V_i} = \frac{ K·\omega_0^2}{s^2 + s · \omega_0/Q + \omega_0^2} $$

En las siguientes secciones se describirá la relación entre los parámetros y los elementos circuitales.

Esta topología fue propuesta originalmente en 1955 por R. P. Sallen and E. L. Key del [laboratorio Lincoln](https://en.wikipedia.org/wiki/MIT_Lincoln_Laboratory), en el MIT [[wiki]](https://en.wikipedia.org/wiki/Sallen%E2%80%93Key_topology). Es uno de los primeros filtros activos publicados, y posiblemente uno de los más populares. Una de sus principales ventajas al momento de su publicación fue la posibilidad de diseñar filtros de orden superior de forma modular. Es decir que cada etapa podía diseñarse por separado debido al uso de seguidores de tensión, o lo que se consideró años después, la topología de filtos *fuente de tensión controlada por tensión* (VCVS por sus siglas en inglés). Otra ventaja importante es que necesita un solo dispositivo activo, en nuestro caso un *opamp*, aunque la publicación original se propuso un amplificador valvular en configuración de cátodo común. La idea original fue que el dispositivo activo funcione aproximadamente como un *buffer*, proveyendo separación de las secciones adyacentes (casi) sin atenuación de la señal. En el caso de la configuración actual, el *opamp* también permite obtener ganancia, como veremos más adelante. Ciertamente esta topología tiene varias desventajas, posiblemente la más notoria es su dificultad de implementar factores de selectividad (Q) elevados, como se analizará en este documento.

<a id='indice'></a>




<a id='1.a'></a>

### 1.a) Análisis teórico de la función transferencia


Se comienza analizando las ecuaciones que gobiernan el comportamiento de la red propuesta:


 1. Nodo $V_b$
 
  $ V_b . (G_1 + G_2 + s.C) - V_i . G_1 - V_a . G_2  - V_o . s.C = 0$


 2. Nodo $V_a$ ó $V_+$
 
  $ -V_b . G_2 + V_a . (G_2 + s.C) = 0$


 3. Nodo $V_-$
 
  $ V_- . (G_a + G_b) - V_o . G_b = 0$


Si consideramos que la red está **realimentada negativamente**, podemos agregar la siguiente ecuación:


 4. $V_- = V_+ = V_a$
 
  
y proceder a resolver el sistema de ecuaciones para $\frac{V_o}{V_i}$, lo cual concluiría el análisis de la transferencia del filtro. 

Sin embargo, a modo de ejemplo, demostraremos las posibilidades de sympy para el manejo de expresiones algebraicas analizando la red como si no supiéramos de qué clase de realimentación se trata. Para tal fin debemos contemplar la siguiente ecuación que describe el funcionamiento del *opamp*.


 4. (bis) $ (V_+ - V_-). A(s) = (V_a - V_-). A(s) = V_o$
 
Luego si asumimos $A(s) \to \infty$ para una salida $V_o$ finita, entonces 4 (bis) se convierte en la ecuación 4. Finalmente, el comportamiento del *opamp* también puede hacerse tan complejo como se desee, por caso usando el modelo de integrador


 5. $ A(s) = \frac{\omega_t}{s}$


Dependiendo la complejidad del análisis podemos optar por cualquiera de los tres caminos sugeridos más arriba. A modo de ejemplo, resolveremos los tres para presentar las posibilidades de sympy.


[Volver al índice](#indice)

In [1]:
# Inicialización e importación de módulos

# Módulos para Jupyter
import warnings
warnings.filterwarnings('ignore')

# Módulos externos
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import TransferFunction

fig_sz_x = 8
fig_sz_y = 6
fig_dpi = 150 # dpi

mpl.rcParams['figure.figsize'] = (fig_sz_x, fig_sz_y)
mpl.rcParams['figure.dpi'] = fig_dpi

plt.figure(1)
plt.plot(np.random.rand(10))
plt.close(1)

# módulo de análisis simbólico
import sympy as sp
# variable de Laplace
from sympy.abc import s
from IPython.display import display, Math, Markdown

# Ahora importamos las funciones de PyTC2
from pytc2.sistemas_lineales import simplify_n_monic, parametrize_sos
from pytc2.general import print_subtitle


<a id='1.b'></a>

### 1.b) Alternativa 1: Red negativamente realimentada 

Procedemos a ingresar las ecuaciones 1-4.

[Volver al índice](#indice)

In [2]:

# variables simbólicas
s = sp.symbols('s', complex=True)
Va, Vb, Vi, Vo = sp.symbols("Va, Vb, Vi, Vo")
G1, G2, Ga, Gb, C, As, wt = sp.symbols("G1, G2, Ga, Gb, C, As, wt")

# Sistemas de ecuaciones del modelo ideal, negativamente realimentado
aa = sp.solve([ 
                Vb*(G1 + G2 + s*C) - Vi * G1 - Va * G2 - Vo * s*C, 
                -Vb*G2 + Va * (G2 + s*C),
                -Vo*Gb + Va * (Ga + Gb)
                ], 
                [Vo, Vi, Va, Vb])
T1 = aa[Vo]/aa[Vi]

num, den = sp.fraction(sp.simplify(sp.expand(T1)))
num = sp.Poly(num,s)
den = sp.Poly(den,s)
num1, den1, w_on, Q_n, w_od, Q_d, k1 = parametrize_sos(num, den)

print_subtitle('Transferencia obtenida')

display(Math( r' \frac{V_o}{V_i} = ' + sp.latex(sp.Mul(k1,num1/den1, evaluate=False)) ))

print_subtitle('Parámetros')

display(Math( r' k = ' + sp.latex(k1) ))

display(Math( r' \omega_0 = ' + sp.latex(w_od) ))

display(Math( r' Q = ' + sp.latex(Q_d) ))


#### Transferencia obtenida

<IPython.core.display.Math object>

#### Parámetros

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<a id='1.c'></a>

### 1.c) Alternativa 2: Sin asumir realimentación negativa (RN) 

Procedemos a ingresar las ecuaciones 1-3 y 4bis.

[Volver al índice](#indice)

In [3]:

# pata inversora del opamp
Vinv  = sp.symbols("Vinv")

# Sistemas de ecuaciones del modelo ideal, negativamente realimentado
aa = sp.solve([ 
                Vb*(G1 + G2 + s*C) - Vi * G1 - Va * G2 - Vo * s*C, 
                -Vb*G2 + Va * (G2 + s*C),
                -Vo*Gb + Vinv * (Ga + Gb),
                (Va - Vinv) * As - Vo
                ], 
                [Vo, Vi, Vinv, Va, Vb])
T2 = aa[Vo]/aa[Vi]

# modelo ideal sin asumir realimentación negativa
T2 = sp.limit(T2, As, sp.oo)

num, den = sp.fraction(sp.simplify(sp.expand(T2)))
num = sp.Poly(num,s)
den = sp.Poly(den,s)
num2, den2, w_on2, Q_n2, w_od2, Q_d2, k2 = parametrize_sos(num, den)

print_subtitle('Transferencia sin asumir RN')

display(Math( r' \frac{V_o}{V_i} = ' + sp.latex(sp.Mul(k2,num2/den2, evaluate=False)) ))

print_subtitle('Parámetros')

display(Math( r' k = ' + sp.latex(k2) ))

display(Math( r' \omega_0 = ' + sp.latex(w_od2) ))

display(Math( r' Q = ' + sp.latex(Q_d2) ))


#### Transferencia sin asumir RN

<IPython.core.display.Math object>

#### Parámetros

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<a id='1.d'></a>

Afortunadamente los resultados convergen, sin embargo, esto no siempre podría ser así. A modo de ejemplo se finaliza la demostración de las posibilidades de sympy extendiendo el análisis de la red considerando al opamp mediante su modelo de integrador.

### 1.d) Alternativa 3: Sin asumir RN y *opamp* no ideal

Procedemos a ingresar las ecuaciones 1-3, 4bis y 5.

[Volver al índice](#indice)

In [4]:

# Sistemas de ecuaciones del modelo ideal, negativamente realimentado
aa = sp.solve([ 
                Vb*(G1 + G2 + s*C) - Vi * G1 - Va * G2 - Vo * s*C, 
                -Vb*G2 + Va * (G2 + s*C),
                -Vo*Gb + Vinv * (Ga + Gb),
                (Va - Vinv) * As - Vo
                ], 
                [Vo, Vi, Vinv, Va, Vb])

T3 = aa[Vo]/aa[Vi]

# modelo integrador A(s)=\omega_t/s (sin asumir realimentación negativa)
T3 = sp.simplify(sp.expand(T3.subs(As, wt/s)))

T3 = simplify_n_monic(T3)

print_subtitle('Transferencia opamp no ideal')

display(Math( r' \frac{V_o}{V_i} = ' + sp.latex(T3) ))


#### Transferencia opamp no ideal

<IPython.core.display.Math object>

Con esto se termina la demostración de las posibilidades de simulación simbólica de sympy.

En el [siguiente documento]() se retoma el análisis de la transferencia para *opamp ideal*, como se analizó en las alternativas 1 y 2. Convendrá recordar la transferencia de tensión definida genéricamente

$$ T(s) = \frac{ K·\omega_0^2}{s^2 + s · \omega_0/Q + \omega_0^2} $$

siendo los parámetros para las alternativas 1 y 2

$$ \omega_0^2=\frac{G_1 G_2}{C^2} $$

$$ K=1+G_a/G_b $$

$$ Q=\frac{ \sqrt{G_1 G_2}}{G_1 + G_2(2-K)} $$

y para $R_1 = \frac{1}{G1} = R_2 = R_a = R $ llegamos a 

$$ \omega_0=\frac{1}{RC} $$

$$ Q=\frac{ 1}{3-K} $$

$$ K=1+R_b/R $$

[Volver al índice](#indice)
